<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/CNN_with_equal_sampling_w_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

11/3/2019

In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "contractions"
MODEL_NAME = "contractions"
MAX_SEQ_LENGTH = 20
NUM_MALE_EXAMPLES = 2000000
NUM_FEMALE_EXAMPLES = 2000000
VAL_SET_SIZE = 1000000
NUM_EPOCHS = 15
BATCH_SIZE = 1000

# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 71kB 4.6MB/s 
  Found existing installation: fsspec 0.5.2
    Uninstalling fsspec-0.5.2:
      Successfully uninstalled fsspec-0.5.2


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [8]:
### here is where we do some simple sampling
male_train_rows = train_df[train_df.op_gender=='M']
female_train_rows = train_df[train_df.op_gender=='W']

male_train_to_use = male_train_rows[:2000000]
female_train_to_use = female_train_rows[:2000000]
print(len(male_train_to_use))
print(len(female_train_to_use))

combined_train_df = pd.concat([male_train_to_use, female_train_to_use])
combined_train_df = combined_train_df.sample(frac=1)
combined_train_df.head()

2000000
2000000


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category
2966190,3739549,55701792,W,125972,Pamela,why do people vote for these idiots?,Debbie Stabenow,Congress_Democratic
7215973,10001381,86386775,M,363021,Gary,you sir are a sellout. i hope you pay with your job come next election. i know you will never ge...,Richard Burr,Congress_Republican
942544,942544,29687946,M,13705,Paula,how are you love you,Rand Paul,Congress_Republican
4821407,6504693,7212873,W,246483,Mike,the dems must be the party of no!,Kamala Harris,Congress_Democratic
5420508,7221128,60239508,W,273482,Rochelle,thank you for attending amy and being the bigger person i thought you were!!! whew! was worrie...,Amy Klobuchar,Congress_Democratic


In [0]:
y_train = combined_train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(combined_train_df.response_text.values)
new_sentences_test = get_text_list(dev_df.response_text.values)

In [13]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 04 minutes, 51 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 20

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [16]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
/ [4 files][  2.1 GiB/  2.1 GiB]  108.3 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
\
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [18]:
# trying to figure out which words are empty here
counter = 0
empty_indexes = []
for index, row in enumerate(embedding_matrix):
  if sum(row) == 0:
    empty_indexes.append(index)
    counter += 1
  if counter > 1000:
    break

for idx in empty_indexes:
  try:
    print(tokenizer.index_word[idx])
  except:
    print("No entry for {}".format(idx))


No entry for 0
stillsanders
youtu
fbid
•
palestinei
🇺🇸
standwithrand
👍
feelthebern
🇺🇸🇺🇸
obummer
huffingtonpost
rinos
❤️
washingtonpost
lmao
killary
╬═╬
doyourjob
fbcdn
bernieorbust
hphotos
😍
neverhillary
hahaha
😊
isil
sheeple
nobillnobreak
repub
bridenstine
berniesanders
trump2016
1073741828
randrally
❤
bengazi
😂
😡
aspx
thehill
drumpf
libtards
cispa
scontent
dailycaller
§
thinkprogress
obozo
sphotos
bernie2016
randpaul
akamaihd
bonamici
stopp
hilliary
kobanê
politicususa
teabaggers
fref
👎
banksters
westernjournalism
agsaf
washingtontimes
libtard
hahahaha
someunknownushistory
nationalreview
2366
koolaid
opensecrets
nodapl
teaparty
notif
imwithher
♥
demandavote
shouldnt
dapl
2fwww
███
🇺🇸🇺🇸🇺🇸
0bama
😉
wtg
washingtonexaminer
cnsnews
imho
nobama
😀
على
occupydemocrats
ovomit
͡°
infowars
♡
nevertrump
oboma
gofundme
worstresponders
govtrack
conservativetribune
devoss
dumbass
daca
repugs
wenstrup
🙏
benifits
latimes
teabagger
democrates
theguardian
dailymail
cromnibus
p71bjjcp8
pnref
abcnews
prot

In [19]:
# hmmmm....
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.23317245077759902

In [20]:
embedding_matrix.shape

(587385, 100)

In [21]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           58738500  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [23]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 4000000 samples, validate on 1000000 samples
Epoch 1/15
4000000/4000000 [==============================] - 106s 26us/sample - loss: 0.5964 - acc: 0.6469 - val_loss: 0.6523 - val_acc: 0.6231
Epoch 2/15
4000000/4000000 [==============================] - 105s 26us/sample - loss: 0.5782 - acc: 0.6633 - val_loss: 0.6459 - val_acc: 0.6252
Epoch 3/15
4000000/4000000 [==============================] - 106s 26us/sample - loss: 0.5722 - acc: 0.6684 - val_loss: 0.6370 - val_acc: 0.6530
Epoch 4/15
4000000/4000000 [==============================] - 104s 26us/sample - loss: 0.5685 - acc: 0.6717 - val_loss: 0.6581 - val_acc: 0.6173
Epoch 5/15
4000000/4000000 [==============================] - 104s 26us/sample - loss: 0.5657 - acc: 0.6743 - val_loss: 0.6560 - val_acc: 0.6245
Epoch 6/15
4000000/4000000 [==============================] - 106s 26us/sample - loss: 0.5636 - acc: 0.6755 - val_loss: 0.6601 - val_acc: 0.6151
Epoch 7/15
4000000/4000000 [==============================] - 108s 27us/samp

In [24]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.6877
Testing Accuracy:  0.6267


In [0]:
preds = model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [29]:
wrong_preds.op_gender.value_counts()

M    689307
W    166556
Name: op_gender, dtype: int64

In [30]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.3782477876337767

In [31]:
#random sample of mistakes
wrong_preds.head(100)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2078802,11219782,77234050,W,398978,Kathy,"was not finished with my post, i want the military to recognized that fort mccellan alabama is a...",Cathy McMorris Rodgers,Congress_Republican,0.571325,M
375560,3125651,23984618,W,82876,Carolyn,i feel better knowing you will get nothing done in this committee based on your history of doing...,Ann Wagner,Congress_Republican,0.627091,M
133815,2289673,89989833,M,49010,Brandi,if only it was for good...,Ted Poe,Congress_Republican,0.473265,W
2153523,11294503,77234050,W,400424,Barb,so tell me again why we should bail them out!,Cathy McMorris Rodgers,Congress_Republican,0.679799,M
1859332,10892260,10458808,M,385772,Josh,#nofeednorevolution,Mark Takano,Congress_Democratic,0.481940,W
...,...,...,...,...,...,...,...,...,...,...
599561,4848615,46506154,M,160191,GiGi,I am sorry i was mean to you marco,Marco Rubio,Congress_Republican,0.480936,W
115294,2271152,89989833,M,48075,Lazaro,it critical,Ted Poe,Congress_Republican,0.441309,W
1013681,5262735,46506154,M,161357,Ron,sounds unamerican to me........,Marco Rubio,Congress_Republican,0.488357,W
2163791,11304771,77234050,W,400987,Tim,semper fi!,Cathy McMorris Rodgers,Congress_Republican,0.825701,M


In [32]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:100]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2064289,11204043,48870736,W,394920,Justin,"if we still had freedom of speech in this country i would give u a piece of my mind, but since w...",Betty McCollum,Congress_Democratic,0.999839,M
2105736,11246716,77234050,W,399421,Rj,the big one is to block the patriot act . . . definitely with rand on this one,Cathy McMorris Rodgers,Congress_Republican,0.999808,M
1963043,11012435,86569077,W,389663,Charlene,"you hit the nail on the head....its the ""american dream"" right? its sad to see how greedy our le...",Kirsten Gillibrand,Congress_Democratic,0.999739,M
2030227,11079619,86569077,W,390346,John,larry north do you mean senator chuck you schumer ?,Kirsten Gillibrand,Congress_Democratic,0.999638,M
1912412,10961804,86569077,W,389338,Kristie,the president's job is to keep this country safe. kirsten gillibrand - have you listened to the...,Kirsten Gillibrand,Congress_Democratic,0.999602,M
...,...,...,...,...,...,...,...,...,...,...
2024791,11074183,86569077,W,390272,John,"thank you mr. obama. i know you will catch he will from republicans for this but i, sir, salute ...",Kirsten Gillibrand,Congress_Democratic,0.995219,M
1923828,10973220,86569077,W,389341,Nancy,"thank you very much for this no vote! now, how about senator schumer?",Kirsten Gillibrand,Congress_Democratic,0.995083,M
2103912,11244892,77234050,W,399380,Mark,let us listen to what louie gohmert has to say about the vote for speaker... http://youtu.be/wkj...,Cathy McMorris Rodgers,Congress_Republican,0.995059,M
2074071,11215051,77234050,W,398699,Sharon,thank you congressman rodgers!! <3 I am ready too <3,Cathy McMorris Rodgers,Congress_Republican,0.995005,M


In [33]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:100]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1870185,10903113,10458808,M,385776,Hannah,i see tulsi!,Mark Takano,Congress_Democratic,0.000002,W
1373869,7792163,99469231,M,296202,Michael,as the terrorist attacks abroad continue....here at home (ignorance) on the behalf of state leve...,John McCain,Congress_Republican,0.000012,W
1099796,5348850,46506154,M,161705,Sarah,I am not even in florida (from wa) and I am so happy for you! i wish that you had been running ...,Marco Rubio,Congress_Republican,0.000031,W
991634,5240688,46506154,M,161317,Laura,"keep the faith ms. ayotte, you are good for new hampshire and our country !! you have more supp...",Marco Rubio,Congress_Republican,0.000035,W
890088,5139142,46506154,M,161007,Kim,good luck scott brown; rooting for you...........jan,Marco Rubio,Congress_Republican,0.000048,W
...,...,...,...,...,...,...,...,...,...,...
1736543,10259680,32570461,M,382048,Sara,"love that photo of hassan, you, and warren! what a dream team!",Tim Kaine,Congress_Democratic,0.000621,W
2204222,11908399,17310,M,440152,Carlos,http://petitions.moveon.org/sign/senator-stabenow-support,Sander Levin,Congress_Democratic,0.000623,W
1330852,7749146,99469231,M,296110,Rhona,i hope that you and sanders continue to work together. you are but a few in congress who are not...,John McCain,Congress_Republican,0.000624,W
238525,2441731,71417411,M,54517,Darkness,thank you congressman farenthold for sharing who you talk to! and thank you elizabeth for your w...,Blake Farenthold,Congress_Republican,0.000626,W


In [0]:
# preds in the middle range
borderline_wrong_preds_f = wrong_preds[wrong_preds.probs.between(.4, .5)]

In [35]:
# this is how many wrong preds were in the "borderline" range
# where model predicted female instead of male
wrong_preds.shape[0] - borderline_wrong_preds_f.shape[0]

503170

In [36]:
borderline_wrong_preds_f.head(40)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
133815,2289673,89989833,M,49010,Brandi,if only it was for good...,Ted Poe,Congress_Republican,0.473265,W
1859332,10892260,10458808,M,385772,Josh,#nofeednorevolution,Mark Takano,Congress_Democratic,0.481940,W
279975,2544510,59227815,M,63426,Elizabeth,paul ryan is nasty! do not let him trample on people! how do you call yourself a christian? ...,Paul Ryan,Congress_Republican,0.494290,W
1817029,10340166,32570461,M,382767,Ron,"joelle, i went by your post.",Tim Kaine,Congress_Democratic,0.418311,W
1531757,7950051,99469231,M,296445,Gary,"by action, I am sure you mean action by those who you want to steal tricare prime from when they...",John McCain,Congress_Republican,0.481477,W
117213,2273071,89989833,M,48153,Alan,"meanwhile, hillary's next release is going to happen on labor day to minimize the coverage and v...",Ted Poe,Congress_Republican,0.464970,W
322227,2586762,59227815,M,63455,Christopher,in other words all the things that trump wants for all americans. good to know that we need to ...,Paul Ryan,Congress_Republican,0.474284,W
1197226,6643410,48816171,M,252913,Cathy,did you strongly oppose bush? or is fighting for oil ok? forget human lives.,Kenny Marchant,Congress_Republican,0.449292,W
1390823,7809117,99469231,M,296238,Craig,https://www.facebook.com/photo.php?fbid=10204358182579439&set=p.10204358182579439&type=1&theater,John McCain,Congress_Republican,0.427914,W
871529,5120583,46506154,M,160911,Martin,napalm...bring it back....drop it on every one of their convoys and outposts....flame throwers a...,Marco Rubio,Congress_Republican,0.476983,W


In [37]:
borderline_wrong_preds_m = wrong_preds[wrong_preds.probs.between(.5, .6)]

# this is how many wrong preds were in the "borderline" range
# where model predicted female instead of male
wrong_preds.shape[0] - borderline_wrong_preds_m.shape[0]

776483

In [38]:
borderline_wrong_preds_m.head(40)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2078802,11219782,77234050,W,398978,Kathy,"was not finished with my post, i want the military to recognized that fort mccellan alabama is a...",Cathy McMorris Rodgers,Congress_Republican,0.571325,M
2087973,11228953,77234050,W,399295,Charlie,obamacare is saving lives and money. why are you so against this? there has room for improveme...,Cathy McMorris Rodgers,Congress_Republican,0.552895,M
2043454,11092846,86569077,W,390560,L,i thought that was gillfords bill..,Kirsten Gillibrand,Congress_Democratic,0.599998,M
2109902,11250882,77234050,W,399524,Debbie,http://www.politico.com/story/2014/03/washington-state-obamacare-no-frills-success-105188.html,Cathy McMorris Rodgers,Congress_Republican,0.556886,M
1913376,10962768,86569077,W,389338,Kurt,why is it ok for previous presidents to invoke this executive order and not alright for trump. i...,Kirsten Gillibrand,Congress_Democratic,0.540804,M
2142570,11283550,77234050,W,400017,Michael,"cathy, as my congress person why is congress and you not investigating the u-tube video of presi...",Cathy McMorris Rodgers,Congress_Republican,0.508375,M
1961805,11011197,86569077,W,389658,Arnold,disagee...you just put american lives at risk by allowing the middle east to continue to control...,Kirsten Gillibrand,Congress_Democratic,0.538763,M
98954,2254812,47977187,W,43686,Karen,exactly,Catherine Cortez Masto,Congress_Democratic,0.537494,M
2119288,11260268,77234050,W,399693,Kathy,i loved him and all he did,Cathy McMorris Rodgers,Congress_Republican,0.553785,M
2062574,11202328,48870736,W,394703,Cheryl,thank you! all !,Betty McCollum,Congress_Democratic,0.507488,M


In [39]:
dev_df.op_gender.value_counts()

M    1948371
W     344536
Name: op_gender, dtype: int64